# 08 - Evaluation & Testing

**Build robust evaluation frameworks for AI systems.**

## Learning Objectives

By the end of this notebook, you will:
- Design evaluation frameworks
- Build test datasets
- Use LLM-as-judge for evaluation
- Implement regression testing

## Table of Contents

1. [Why Evaluation Matters](#why)
2. [Building Test Datasets](#datasets)
3. [Automated Evaluation](#auto)
4. [LLM-as-Judge](#llm-judge)
5. [Regression Testing](#regression)
6. [Exercises](#exercises)
7. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
import json
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

print("Setup complete!")

---
## 1. Why Evaluation Matters <a id='why'></a>

### Challenges with AI Systems:
- **Non-deterministic**: Same input can give different outputs
- **Subjective quality**: "Good" is hard to define
- **Emergent behavior**: Changes can have unexpected effects

### Evaluation Types:
```
Type              Use Case                   Example Metrics
──────────────────────────────────────────────────────────────
Exact Match       Classification, QA         Accuracy, F1
Similarity        Text generation            BLEU, ROUGE
Semantic          Open-ended generation      LLM-as-Judge
Task-specific     RAG, Agents                Precision@K, Success rate
```

---
## 2. Building Test Datasets <a id='datasets'></a>

In [ ]:
# GUIDED: Create test cases
from src.evaluation import TestCase, Evaluator

# Define test cases
test_cases = [
    TestCase(
        id="math_1",
        input="What is 2 + 2?",
        expected="4",
        metadata={"category": "math", "difficulty": "easy"}
    ),
    TestCase(
        id="math_2",
        input="What is the square root of 144?",
        expected="12",
        metadata={"category": "math", "difficulty": "medium"}
    ),
    TestCase(
        id="fact_1",
        input="What is the capital of France?",
        expected="Paris",
        metadata={"category": "geography", "difficulty": "easy"}
    ),
    TestCase(
        id="fact_2",
        input="Who wrote Romeo and Juliet?",
        expected="Shakespeare",
        metadata={"category": "literature", "difficulty": "easy"}
    ),
]

print(f"Created {len(test_cases)} test cases")
for tc in test_cases:
    print(f"  [{tc.id}] {tc.input[:40]}...")

In [ ]:
# GUIDED: Save and load test cases
import json
from pathlib import Path

# Save test cases
test_data = [
    {
        "id": tc.id,
        "input": tc.input,
        "expected": tc.expected,
        "metadata": tc.metadata
    }
    for tc in test_cases
]

test_file = Path("../data/test_cases.json")
test_file.parent.mkdir(parents=True, exist_ok=True)

with open(test_file, "w") as f:
    json.dump(test_data, f, indent=2)

print(f"Saved to {test_file}")

# Load test cases
evaluator = Evaluator()
evaluator.add_tests_from_json(str(test_file))

---
## 3. Automated Evaluation <a id='auto'></a>

In [ ]:
# GUIDED: Run automated evaluation
from src.evaluation import Evaluator
from src.llm_utils import LLMClient

# Create evaluator and add tests
evaluator = Evaluator()
for tc in test_cases:
    evaluator.add_test(tc.id, tc.input, tc.expected, **tc.metadata)

# Define the system to test
client = LLMClient(provider="openai", model="gpt-4o-mini")

def qa_system(question: str) -> str:
    return client.chat(
        message=question,
        system="Answer the question concisely in one or two words when possible."
    )

# Run evaluation
results = evaluator.run(qa_system)

print("\n" + "="*50)
print(results.summary())
print("="*50 + "\n")

for r in results.results:
    status = "PASS" if r.passed else "FAIL"
    print(f"[{status}] {r.id}: '{r.actual}' (expected: '{r.expected}')")

In [ ]:
# GUIDED: Custom evaluation function
def fuzzy_match_evaluator(
    input: str,
    actual: str,
    expected: str
) -> tuple[bool, float, str]:
    """Fuzzy matching that handles variations."""
    actual_clean = actual.strip().lower()
    expected_clean = expected.strip().lower()
    
    # Exact match
    if actual_clean == expected_clean:
        return True, 1.0, "Exact match"
    
    # Expected in actual
    if expected_clean in actual_clean:
        return True, 0.9, "Expected found in response"
    
    # Partial match (for numbers, names, etc.)
    actual_words = set(actual_clean.split())
    expected_words = set(expected_clean.split())
    
    if expected_words & actual_words:
        overlap = len(expected_words & actual_words) / len(expected_words)
        return overlap >= 0.5, overlap, f"Partial match ({overlap:.0%})"
    
    return False, 0.0, f"No match: expected '{expected}', got '{actual}'"

# Run with custom evaluator
results = evaluator.run(qa_system, evaluator_fn=fuzzy_match_evaluator)
print(results.summary())

---
## 4. LLM-as-Judge <a id='llm-judge'></a>

In [ ]:
# GUIDED: Use LLM to evaluate outputs
from src.evaluation import LLMJudge
from src.llm_utils import LLMClient

llm = LLMClient(provider="openai", model="gpt-4o-mini")
judge = LLMJudge(llm)

# Evaluate a response
question = "Explain what machine learning is in simple terms."
answer = """Machine learning is when computers learn from examples instead of 
being explicitly programmed. It's like how a child learns to recognize cats 
by seeing many pictures of cats."""

score, feedback = judge.evaluate(
    question=question,
    answer=answer,
    criteria="accuracy, clarity, simplicity, completeness"
)

print(f"Question: {question}")
print(f"\nAnswer: {answer}")
print(f"\nScore: {score:.2f}")
print(f"Feedback: {feedback}")

In [ ]:
# GUIDED: Compare two responses
question = "What are the benefits of exercise?"

answer_a = "Exercise is good for you. It helps with fitness."

answer_b = """Regular exercise offers numerous benefits including improved 
cardiovascular health, better mental well-being, weight management, 
increased energy levels, and reduced risk of chronic diseases."""

winner, explanation = judge.compare(question, answer_a, answer_b)

print(f"Question: {question}\n")
print(f"Answer A: {answer_a}\n")
print(f"Answer B: {answer_b}\n")
print(f"Winner: {winner}")
print(f"Explanation: {explanation}")

In [ ]:
# GUIDED: Create LLM-based evaluator function
def llm_evaluator(input: str, actual: str, expected: str) -> tuple[bool, float, str]:
    """Use LLM to evaluate responses."""
    score, feedback = judge.evaluate(
        question=input,
        answer=actual,
        criteria="accuracy, relevance",
        reference=expected
    )
    return score >= 0.7, score, feedback

# Test open-ended questions
open_evaluator = Evaluator()
open_evaluator.add_test(
    id="explain_1",
    input="What is photosynthesis?",
    expected="The process by which plants convert sunlight into energy."
)

def explain_system(question: str) -> str:
    return client.chat(
        message=question,
        system="Explain concepts clearly and concisely."
    )

results = open_evaluator.run(explain_system, evaluator_fn=llm_evaluator)
print(results.summary())
print(f"\nFeedback: {results.results[0].feedback}")

---
## 5. Regression Testing <a id='regression'></a>

In [ ]:
# GUIDED: Create regression test suite
from dataclasses import dataclass, field
from datetime import datetime
import json

@dataclass
class RegressionSuite:
    """Manage regression tests across versions."""
    
    name: str
    tests: list[dict] = field(default_factory=list)
    history: list[dict] = field(default_factory=list)
    
    def add_test(self, id: str, input: str, baseline_output: str):
        """Add a test with baseline output."""
        self.tests.append({
            "id": id,
            "input": input,
            "baseline": baseline_output
        })
    
    def run(self, system_fn, threshold: float = 0.8):
        """Run regression tests."""
        results = []
        
        for test in self.tests:
            output = system_fn(test["input"])
            
            # Compare with baseline using similarity
            score, feedback = judge.evaluate(
                question=test["input"],
                answer=output,
                criteria="semantic similarity",
                reference=test["baseline"]
            )
            
            results.append({
                "id": test["id"],
                "output": output,
                "baseline": test["baseline"],
                "score": score,
                "regressed": score < threshold
            })
        
        # Record run
        run_result = {
            "timestamp": datetime.now().isoformat(),
            "results": results,
            "regressions": sum(1 for r in results if r["regressed"])
        }
        self.history.append(run_result)
        
        return run_result
    
    def save(self, path: str):
        """Save suite to file."""
        with open(path, "w") as f:
            json.dump({
                "name": self.name,
                "tests": self.tests,
                "history": self.history
            }, f, indent=2)

# Create regression suite
suite = RegressionSuite(name="QA Regression")
suite.add_test(
    id="qa_1",
    input="What is the capital of Japan?",
    baseline_output="Tokyo"
)
suite.add_test(
    id="qa_2",
    input="What is 10 * 5?",
    baseline_output="50"
)

print(f"Created regression suite with {len(suite.tests)} tests")

In [ ]:
# GUIDED: Run regression tests
run_result = suite.run(qa_system)

print(f"Regression Run: {run_result['timestamp']}")
print(f"Regressions found: {run_result['regressions']}\n")

for r in run_result['results']:
    status = "REGRESSION" if r['regressed'] else "OK"
    print(f"[{status}] {r['id']}: score={r['score']:.2f}")
    if r['regressed']:
        print(f"   Baseline: {r['baseline']}")
        print(f"   Current:  {r['output']}")

---
## 6. Exercises <a id='exercises'></a>

### Exercise 1: Build Test Suite

Create a comprehensive test suite for a RAG system.

In [ ]:
# TODO: Create test cases covering: factual accuracy, relevance, completeness

# Your code here:


### Exercise 2: Custom Metrics

Implement domain-specific evaluation metrics.

In [ ]:
# TODO: Create metrics for code generation (syntax, correctness, style)

# Your code here:


---
## 7. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You can build test datasets
- [ ] You implemented automated evaluation
- [ ] You used LLM-as-Judge
- [ ] You understand regression testing

### Next Steps

In the next notebook, we'll cover **Production Deployment** - taking AI apps to production!